### Use Case: Sports & Fitness E-commerce Product Assistant
Build AI Agent that helps customers to find sports and fitness products, compare prices, analyze product ratings, and provide personalized recommendations from the e-commerce dataset.

### 🧩 Agents as Tools

In "Agents as Tools"  Pattern , the primary agent can invoke another agent like a tool, receive its output, and continue the conversation smoothly. The user never experiences a speaker switch. The invoked agent:

- Runs independently

- Performs one focused task

- Returns results as data

- Does not interact with the user directly

The main agent remains the single voice throughout.

### 🏀 Example — Sports Product Assistant

Consider your SportsOrchestrator agent. It helps users shop for sports equipment. Behind the scenes, it collaborates with multiple expert agents:

- ProductSearchAgent — finds products in a category

- PricingAgent — filters products by price range

- DealsAgent — finds discounted items

- RecommendationAgent — suggests the best options

Using the Agents as Tools pattern, the orchestrator never hands over the conversation. Instead, it does something like this:

#### User: Show me the best badminton rackets under ₹2000

- (1) The orchestrator analyzes intent: - Needs a sports item ,  - Must respect a budget

- (2) It calls PricingAgent as a tool:
    → find_products_by_price_range("Sports", 0, 2000)

- (3) PricingAgent returns structured product data.

- (4) The orchestrator summarizes the results
    and replies in natural language.


At no point does the PricingAgent speak to the user. It's used just like a function call, even though it's a fully capable agent.
### 🎯 Why Use "Agents as Tools" Pattern?

| **Benefit**        | **Description**                                       |
|-------------------|-------------------------------------------------------|
| Consistent voice   | Only one agent talks to the user                      |
| Modularity         | Each specialist agent solves one problem              |
| Reusability        | Agents become callable services                       |
| Composability      | Multiple agents can be invoked in one response        |



This design is ideal for assistants that need complex internal reasoning—like an online shopping guide—without overwhelming the user or shifting the conversation context.

In [1]:
#! pip install openai-agents

1️⃣ Import Modules and Load Environment

In [2]:
from agents import Agent, Runner, function_tool
import pandas as pd
import json
import numpy as np
from dotenv import load_dotenv

load_dotenv()

True

### Load the E-Commerce Dataset

In [20]:
# Load the dataset

def load_sports_data():
    df = pd.read_csv('./dataset1.csv')

    df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)


    # Convert formatted price strings ("1,299") into float 1299.0
    df['selling_price'] = df['selling_price'].astype(str).str.replace(',', '').astype(float)
    df['mrp'] = df['mrp'].astype(str).str.replace(',', '').astype(float)

    # Convert ratings; invalid entries become NaN
    df['product_rating'] = pd.to_numeric(df['product_rating'], errors='coerce')
    df['seller_rating'] = pd.to_numeric(df['seller_rating'], errors='coerce')

    # Discount% = (MRP - Selling Price) / MRP
    df['discount_percentage'] = ((df['mrp'] - df['selling_price']) / df['mrp'] * 100).round(2)

    

    return df


In [21]:
# Load data once

sports_df = load_sports_data()
sports_df.head()  # Displays first few records so we can visually validate data

,category_1,category_2,category_3,title,product_rating,selling_price,mrp,seller_name,seller_rating,description,highlights,discount_percentage
0,Sports Books and More,Sports,Cricket,ITWOSERVICES CRICKET NET 100X10 CRICKET NET NY...,4.4,1615.0,4000.0,I2SERVICES,4.4,NaN,Cricket Practice Net NYLON HDPE Material W x H...,59.62
1,Sports Books and More,Sports,Cricket,ITWOSERVICES CRICKET NET GROUND BOUNDARY NET 1...,4.4,152.0,600.0,I2SERVICES,4.4,10 X 10 GREEN CRICKET NET HDPE NYLON.,Cricket HDPE NYLON Material W x H x D: 3.048 x...,74.67
2,Sports Books and More,Sports,Cricket,VICTORY Medium Weight ( Pack of 1 ) Rubber Cri...,3.7,59.0,199.0,VictoryOutlets,4.7,NaN,Cricket Rubber Ball Weight: 110 g,70.35
3,Sports Books and More,Sports,Cricket,LYCAN Junior Cricket Bat Size 3 For Age Group ...,3.9,249.0,1000.0,sellguru,4.8,Lycan Junior Cricket Bat Size 3 For Age Group ...,Age Group 8 Yrs Blade Made of PVC/Plastic Begi...,75.10
4,Sports Books and More,Sports,Cricket,Star X Thrill Fox Heavy Duty First Grade HD Pl...,4.0,349.0,749.0,STARX,4.5,"StarX Brings you ""Thrill Fox Heavy Duty First ...",Age Group 15+ Yrs Blade Made of PVC/Plastic Be...,53.40


### 4️⃣ Define Dataset Tools

These functions are executed by agents, NOT humans.

Each one converts a natural language request into a structured response.


**TOOL FUNCTIONS**

Tools behave like API endpoints. The orchestration agent will call them when the user asks something relevant.
Every tool:
  - Accepts typed parameters
  - Executes pandas filtering logic
  - Returns JSON so the assistant can summarize results to the user




In [5]:
@function_tool
def resolve_category(query: str) -> str:
    """
    Identify product category and subcategory from a natural language query.
    If not found, returns null values for routing clarification.
    """
    #print("resolve_category",query.lower() )
    text = query.lower()
    category = None
    subcategory = None

    # Loop through known categories
    for cat in sports_df['category_2'].unique():
        #if cat.lower() in text:
        
        category = cat.strip()
        # Identify subcategory under the found category
        subcats = sports_df[sports_df['category_2'] == cat]['category_3'].unique()
        print(cat, subcats)
        for sub in subcats:
            sub = sub.strip()
            if isinstance(sub, str) and sub.lower() in text:
                subcategory = sub
                break
        break
    print({"category": category, "subcategory": subcategory})
    return json.dumps({"category": category, "subcategory": subcategory})

In [6]:
# SQL-like query tools for the dataset
@function_tool
def search_products_by_category(category: str, subcategory: str = None) -> str:
    """Search products by category and optional subcategory."""
    print("search_products_by_category", "category" + category, "subcategory"+subcategory)
    try:
        if subcategory:
            results = sports_df[
                (sports_df['category_2'].str.lower().str.strip() == category.lower().strip()) & 
                (sports_df['category_3'].str.lower().str.strip() == subcategory.lower().strip())
            ]
        else:
            results = sports_df[sports_df['category_2'].str.lower().str.strip() == category.lower().strip()]
            
        #print("results",results)
        if results.empty:
            return json.dumps({
                "status": "success",
                "message": f"No products found in {category}" + (f" and {subcategory}" if subcategory else ""),
                "results": []
            }, indent=2)

        
        # Select relevant columns and limit results
        output = results[['title', 'selling_price', 'mrp', 'discount_percentage', 
                         'product_rating', 'seller_name', 'seller_rating']].head(10)
        
        response = {
            "status": "success",
            "total_products": len(results),
            "category": category,
            "subcategory": subcategory,
            "products": output.to_dict('records')
        }
        return json.dumps(response, indent=2)
        
    except Exception as e:
        return json.dumps({
            "status": "error",
            "message": f"Error searching products: {str(e)}"
        }, indent=2)


In [7]:
@function_tool
def find_products_by_price_range(sub_category: str, min_price: float, max_price: float) -> str:
    """Find products in a specific sub_category within a price range."""
    print("find_product_by_price", sub_category , min_price , max_price)
    try:
        results = sports_df[
            (sports_df['category_3'].str.lower().str.strip() == sub_category.lower().strip()) &
            (sports_df['selling_price'] >= min_price) &
            (sports_df['selling_price'] <= max_price)
        ].sort_values('selling_price')
        #print(results)
        if results.empty:
            return json.dumps({
                "status": "success",
                "message": f"No products found in {sub_category} between ₹{min_price} and ₹{max_price}",
                "results": []
            }, indent=2)
        
        output = results[['title', 'selling_price', 'mrp', 'discount_percentage', 
                         'product_rating', 'seller_name']].head(10)
        
        response = {
            "status": "success",
            "total_products": len(results),
            "price_range": f"₹{min_price} - ₹{max_price}",
            "products": output.to_dict('records')
        }
        return json.dumps(response, indent=2)
        
    except Exception as e:
        return json.dumps({
            "status": "error",
            "message": f"Error finding products: {str(e)}"
        }, indent=2)


In [8]:
@function_tool
def get_best_rated_products(sub_category: str, min_rating: float = 4.0) -> str:
    """Get the best rated products in a sub_category."""
    print("get best rated product" , category , min_rating)
    try:
        results = sports_df[
            (sports_df['category_3'].str.lower().str.strip() == category.lower().strip()) &
            (sports_df['product_rating'] >= min_rating)
        ].sort_values('product_rating', ascending=False)
        
        if results.empty:
            return json.dumps({
                "status": "success",
                "message": f"No products found in {sub_category} with rating ≥ {min_rating}",
                "results": []
            }, indent=2)
        
        output = results[['title', 'product_rating', 'selling_price', 'mrp', 
                         'discount_percentage', 'seller_name']].head(10)
        
        response = {
            "status": "success",
            "total_products": len(results),
            "minimum_rating": min_rating,
            "top_rated_products": output.to_dict('records')
        }
        return json.dumps(response, indent=2)
        
    except Exception as e:
        return json.dumps({
            "status": "error",
            "message": f"Error getting rated products: {str(e)}"
        }, indent=2)


In [9]:
@function_tool
def get_best_deals(sub_category: str, min_discount: float = 20.0) -> str:
    """Get products with the highest discounts in a category."""
    print("get_best_deal",category , min_discount)
    try:
        results = sports_df[
            (sports_df['category_3'].str.lower().str.strip() == sub_category.lower().strip()) &
            (sports_df['discount_percentage'] >= min_discount)
        ].sort_values('discount_percentage', ascending=False)
        #print(results)
        if results.empty:
            return json.dumps({
                "status": "success",
                "message": f"No products found in {sub_category} with discount ≥ {min_discount}%",
                "results": []
            }, indent=2)
        
        output = results[['title', 'discount_percentage', 'selling_price', 'mrp', 
                         'product_rating', 'seller_name']].head(10)
        
        response = {
            "status": "success",
            "total_products": len(results),
            "minimum_discount": f"{min_discount}%",
            "best_deals": output.to_dict('records')
        }
        return json.dumps(response, indent=2)
        
    except Exception as e:
        return json.dumps({
            "status": "error",
            "message": f"Error getting deals: {str(e)}"
        }, indent=2)


In [10]:
@function_tool
def compare_sellers(category: str) -> str:
    """Compare sellers in a category based on ratings and number of products."""
    print("compare_sellers" , category)
    
    try:
        seller_stats = sports_df[sports_df['category_2'].str.lower().str.strip() == category.lower().strip()].groupby('seller_name').agg({
            'title': 'count',
            'seller_rating': 'mean',
            'product_rating': 'mean',
            'selling_price': 'mean'
        }).round(2).reset_index()
        
        seller_stats = seller_stats.rename(columns={
            'title': 'product_count',
            'seller_rating': 'avg_seller_rating',
            'product_rating': 'avg_product_rating'
        }).sort_values('avg_seller_rating', ascending=False)
        
        response = {
            "status": "success",
            "category": category,
            "seller_comparison": seller_stats.to_dict('records')
        }
        #print(response)
        return json.dumps(response, indent=2)
        
    except Exception as e:
        print(e)
        return json.dumps({
            "status": "error",
            "message": f"Error comparing sellers: {str(e)}"
        }, indent=2)

In [11]:
@function_tool
def get_available_categories() -> str:
    """Get all available product categories in the dataset."""
    print("get_avl_cat")
    try:
        categories = sports_df['category_2'].unique().tolist()
        subcategories = sports_df.groupby('category_2')['category_3'].unique().to_dict()
        
        response = {
            "status": "success",
            "available_categories": categories,
            "subcategories": subcategories
        }
        return json.dumps(response, indent=2)
        
    except Exception as e:
        return json.dumps({
            "status": "error",
            "message": f"Error getting categories: {str(e)}"
        }, indent=2)


***HOW RECOMMENDATION WORKS:***

 Score = Weighted sum of:
      - 60% → product_rating        (users trust this most)
      - 20% → discount_percentage  (good deals influence conversions)
      - 20% → price optimization   (cheaper products rewarded nonlinearly)

 Why 1 / log(price)? Because:
   - Lower price should boost score
   - But avoid exaggerating extremely cheap products

 RESULT: A ranking that feels like Amazon/Flipkart

In [12]:


def recommend_products(subcategory: str, top_n: int = 5):
    print("recommed_prod", "subcategory" + subcategory, "top_n" + str(top_n))
    df = sports_df[sports_df['category_3'].str.lower().str.strip() == subcategory.lower().strip()]
    #print("df", df)
    if df.empty:
        return {"status": "error", "message": f"No products in {category}"}

    df.loc[:, "score"] = (
        df["product_rating"] * 0.6 +
        (df["discount_percentage"] / 100) * 0.2 +
        (1 / np.log1p(df["selling_price"])) * 0.2
    )
    #print(df)

    top_df = df.sort_values("score", ascending=False).head(top_n)
    

    return {
        "status": "success",
        "category": subcategory,
        "recommendations": top_df[['title','selling_price','mrp','product_rating','discount_percentage']].to_dict('records')
    }

@function_tool
def get_recommendations(subcategory: str) -> str:
    return json.dumps(recommend_products(subcategory), indent=2)

In [13]:
#recommend_products("Badminton")

### 6️⃣ Create Specialist Agents

 Each agent handles ONE domain:
- ProductSearchAgent → Browsing products
- PricingAgent → Budget/search by price
- RatingsAgent → High-rated items
- DealsAgent → Discounts and offers
- SellerComparisonAgent → Seller stats
- CategoryAgent → Dataset exploration
- RecommendationAgent → AI-based suggestions

 These are NOT exposed to users directly. They are wrapped into "tools" for the orchestrator.

In [14]:


resolve_category_agent = Agent("ResolveCategoryAgent","Extract category and subcategory", [resolve_category])
product_search_agent = Agent("ProductSearchAgent","Search sports products", [search_products_by_category])
pricing_agent = Agent("PricingAgent","Handle price queries", [find_products_by_price_range])
ratings_agent = Agent("RatingsAgent","Fetch best rated items", [get_best_rated_products])
deals_agent = Agent("DealsAgent","Show discounted products", [get_best_deals])
seller_agent = Agent("SellerComparisonAgent","Compare sellers", [compare_sellers])
category_agent = Agent("CategoryAgent","Return category list", [get_available_categories])
recommendation_agent = Agent("RecommendationAgent","Recommend best buys", [get_recommendations])

In [15]:
orchestrator_agent = Agent(
    name="SportsOrchestrator",
    instructions="""
You are a sports e-commerce assistant.

You always answer by calling one of the tools.
You never answer directly from your own knowledge.

TOOLS:
- product_search: show or listor search products by category and optional subcategory
- price_filter: filter products within a price range for a category
- top_rated: get best rated products under a category
- best_deals: find highly discounted products in a category
- compare_sellers: compare sellers by category
- categories: list available sports categories
- recommend_products: recommend best products to buy

When the category and subcategory are included in the user message
(e.g., [CATEGORY=Sports; SUBCATEGORY=Cricket]), you MUST use those values 
when calling tools, instead of trying to infer them again.
""",
    tools=[
        product_search_agent.as_tool("product_search", "Search sports products"),
        pricing_agent.as_tool("price_filter", "Filter products within a category by price"),
        ratings_agent.as_tool("top_rated", "Top-rated sports products within a category"),
        deals_agent.as_tool("best_deals", "Best discount products within a category"),
        seller_agent.as_tool("compare_sellers", "Compare  sellers within a category"),
        category_agent.as_tool("categories", "Available sports categories"),
        recommendation_agent.as_tool("recommend_products", "Recommend sports products"),
    ],

    
)


In [16]:
def resolve_category_plain(query: str) -> dict:
    """
    Identify category and subcategory from a natural language query.
    Used in Python (not as a tool) before calling the orchestrator.
    """
    text = query.lower()
    category = None
    subcategory = None

    # Subcategory-first resolution
    for sub in sports_df['category_3'].dropna().unique():
        sub_clean = str(sub).strip()
        if sub_clean.lower() in text:
            subcategory = sub_clean
            # infer category from this subcategory
            cat = sports_df[sports_df['category_3'] == sub]['category_2'].iloc[0]
            category = str(cat).strip()
            return {"category": category, "subcategory": subcategory}

    # Category match
    for cat in sports_df['category_2'].unique():
        cat_clean = str(cat).strip()
        if cat_clean.lower() in text:
            category = cat_clean
            return {"category": category, "subcategory": None}

    return {"category": None, "subcategory": None}


In [17]:
queries = [
    "Show me cricket products",
    "Recommend me best badminton items",
    "Compare sports sellers",
    "What are the cheapest football items under 500?",
    "Give me best deals in cycling"
]

In [18]:

#os.environ["DEBUG"] = "1"
for q in queries:
    print("\n\n")

    # 1️⃣ Resolve category/subcategory in Python
    cat_info = resolve_category_plain(q)
    print(cat_info)
    category = cat_info["category"]
    subcategory = cat_info["subcategory"]

    # 2️⃣ Build an augmented query with explicit context
    if category:
        context_prefix = f"[CATEGORY={category}; SUBCATEGORY={subcategory}] "
    else:
        context_prefix = ""

    augmented_query = context_prefix + q

    # 3️⃣ Call the orchestrator agent
    result = await Runner.run(orchestrator_agent, q)
    
    print("USER:", q)

    print(result)
    print("ASSISTANT:", result.final_output)




{'category': 'Sports', 'subcategory': 'Cricket'}
search_products_by_category categorySports subcategoryCricket
USER: Show me cricket products
RunResult:
- Last agent: Agent(name="SportsOrchestrator", ...)
- Final output (str):
    Here are some cricket products available:
    
    1. ITWOSERVICES CRICKET NET 100X10 CRICKET NET NYLON HDPE (Green)
       - Price: ₹1,615 (MRP: ₹4,000, 59.62% off)
       - Rating: 4.4
    
    2. ITWOSERVICES CRICKET NET GROUND BOUNDARY NET 10X10 FEET (Green)
       - Price: ₹152 (MRP: ₹600, 74.67% off)
       - Rating: 4.4
    
    3. VICTORY Medium Weight Rubber Cricket Tennis Ball (Pack of 1)
       - Price: ₹59 (MRP: ₹199, 70.35% off)
       - Rating: 3.7
    
    4. LYCAN Junior Cricket Bat Size 3 (8 Years, PVC/Plastic, 400g)
       - Price: ₹249 (MRP: ₹1,000, 75.1% off)
       - Rating: 3.9
    
    5. Star X Thrill Fox Heavy Duty First Grade HD Plastic Cricket Bat (1kg)
       - Price: ₹349 (MRP: ₹749, 53.4% off)
       - Rating: 4.0
    
    6. 

C:\Users\vyanktesh.l\AppData\Local\Temp\ipykernel_38192\4146856714.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "score"] = (


USER: Recommend me best badminton items
RunResult:
- Last agent: Agent(name="SportsOrchestrator", ...)
- Final output (str):
    Here are some of the best recommended badminton items:
    
    1. Nongi Badminton Feather Shuttlecock (Pack of 2, Medium 77)
       - Price: ₹69 (MRP: ₹400, 82.75% off)
       - Rating: 4.0
    
    2. LI-NING XP-80-IV Grey Blue Strung Badminton Racquet (Pack of 2, 86g)
       - Price: ₹749 (MRP: ₹1980, 62.17% off)
       - Rating: 4.0
    
    3. LI-NING XP-80-IV Grey Blue Strung Badminton Racquet (Pack of 1, 86g)
       - Price: ₹399 (MRP: ₹990, 59.7% off)
       - Rating: 4.0
    
    4. Adrenex by Flipkart R501 Full Graphite Badminton Racquet (Black Orange, Pack of 1, 90g)
       - Price: ₹949 (MRP: ₹1375, 30.98% off)
       - Rating: 4.1
    
    5. Monika Sports Badminton Kit (2 Racquets, 3 Feather Shuttles & 1 Racquet Cover)
       - Price: ₹299 (MRP: ₹899, 66.74% off)
       - Rating: 3.9
    
    If you want more information or help choosing, let me

# Enabling Session 

In [25]:
from agents import SQLiteSession
    
session = SQLiteSession("customer_123", "sports_conversations.db")


print("🎯 Interactive Sports Shopping Assistant")
print("Type 'quit' to exit\n")

while True:
    query = input("You: ").strip()
    if query.lower() in ['quit', 'exit', 'q']:
        print("Thank you for shopping with us! 🏆")
        break

    if not query:
        continue

    result = await Runner.run(orchestrator_agent, query, session=session)
    print(f"Assistant: {result.final_output}\n")

🎯 Interactive Sports Shopping Assistant
Type 'quit' to exit



You:  show me comparison between bestsellers


Assistant: To compare bestsellers among cricket products, could you specify if you're interested in particular categories like bats, balls, protective gear, or kits? This will help me provide a relevant and focused comparison among top sellers. 

Let me know your preference (e.g., cricket bats, kits, or protective equipment), or I can compare general cricket bestsellers if that works for you!



You:  cricket


Assistant: I can help you compare the top sellers for cricket equipment. Could you specify if you’re looking to compare sellers for particular items like bats, balls, kits, or protective gear? This will help provide a focused comparison.

If you want a general comparison of all best-selling cricket products and their sellers, I’ll proceed with that. Please confirm or specify your preference!



...

Assistant: There seems to be an issue fetching a direct comparison of sellers for all cricket equipment. However, I can help you with top-rated products, best deals, or recommendations for each cricket equipment category (bats, balls, kits, gloves, etc.) to give you a comprehensive view.

Would you like me to show you the best-rated or top deals for each type of cricket equipment? Please specify if you'd like details for a particular subcategory, or I can display all main categories for you to choose from!



You:  show me how the best-rated or top deals for each type of cricket equipment?


get best rated product Sports 4.0
get best rated product Sports 4.0
get best rated product Sports 4.0
get_best_deal Sports 20.0
get_best_deal Sports 20.0
get_best_deal Sports 20.0
get_best_deal Sports 20.0
get best rated product Sports 4.0
Assistant: Currently, there are no highly rated (4.0+) or heavily discounted (20%+ off) cricket bats, balls, kits, or gloves available. This could be due to limited stock or a temporary lack of top-reviewed or sale items in these categories.

Would you like to see all available products in a certain category regardless of rating/discount, or would you prefer recommendations based on popularity or price? Let me know how you'd like to proceed!



You:  exit


Thank you for shopping with us! 🏆
